In [1]:
import pandas as pd 
import xml.etree.ElementTree as et 

xtree = et.parse("Stadiums.xml")
xroot = xtree.getroot() 

df_cols = ["stadiumID","stadium_name", "state", "country", "latitude", "longitude", "capacity"]
rows = []


for node in xroot:
    s_ID = node.find("StadiumID").text if node is not None else None
    s_name = node.find("Name").text if node is not None else None
    s_state = node.find("State").text if node is not None else None
    s_country = node.find("Country").text if node is not None else None
    s_cap = node.find("Capacity").text if node is not None else None
    s_lat = node.find("GeoLat").text if node is not None else None
    s_lon = node.find("GeoLong").text if node is not None else None

    rows.append({"stadiumID":s_ID,"stadium_name": s_name, 
                 "state": s_state, "country":s_country,
                 "latitude":s_lat, "longitude":s_lon, "capacity": s_cap})

out_df = pd.DataFrame(rows, columns = df_cols)

In [2]:
out_df.head()

,stadiumID,stadium_name,state,country,latitude,longitude,capacity
0,1,New Era Field,NY,USA,42.773758,-78.786837,73079
1,2,Hard Rock Stadium,FL,USA,25.957740,-80.238781,76100
2,3,MetLife Stadium,NJ,USA,40.813611,-74.074444,82500
3,4,Gillette Stadium,MA,USA,42.090866,-71.264244,68756
4,5,Paul Brown Stadium,OH,USA,39.095309,-84.516003,65535


In [3]:
import pandas as pd 
import xml.etree.ElementTree as et 

xtree = et.parse("Teams.xml")
xroot = xtree.getroot() 

df_cols = ["key","ID","Team","Name","conference","division","full_name","stadiumID"]
rows = []


for node in xroot.findall('Team'):
    
    t_key = node.find("Key").text if node is not None else None
    t_TeamID = node.find("TeamID").text if node is not None else None
    t_city = node.find("City").text if node is not None else None
    t_name = node.find("Name").text if node is not None else None
    t_conf = node.find("Conference").text if node is not None else None
    t_div = node.find("Division").text if node is not None else None
    t_full = node.find("FullName").text if node is not None else None
    t_s_ID = node.find("StadiumID").text if node is not None else None
    rows.append({"key": t_key, "ID": t_TeamID, 
             "Team": t_city, "Name":t_name,
             "conference":t_conf, "division":t_div, "full_name": t_full, "stadiumID":t_s_ID})
team_df = pd.DataFrame(rows, columns = df_cols)

In [4]:
team_df.head()

,key,ID,Team,Name,conference,division,full_name,stadiumID
0,ARI,1,Arizona,Cardinals,NFC,West,Arizona Cardinals,29
1,ATL,2,Atlanta,Falcons,NFC,South,Atlanta Falcons,45
2,BAL,3,Baltimore,Ravens,AFC,North,Baltimore Ravens,7
3,BUF,4,Buffalo,Bills,AFC,East,Buffalo Bills,1
4,CAR,5,Carolina,Panthers,NFC,South,Carolina Panthers,26


In [5]:
teams_stadium = pd.merge(team_df,
                 out_df,
                 on='stadiumID')
teams_stadium.head()

,key,ID,Team,Name,conference,division,full_name,stadiumID,stadium_name,state,country,latitude,longitude,capacity
0,ARI,1,Arizona,Cardinals,NFC,West,Arizona Cardinals,29,U of Phoenix Stadium,AZ,USA,33.527558,-112.263036,63400
1,ATL,2,Atlanta,Falcons,NFC,South,Atlanta Falcons,45,Mercedes-Benz Stadium,GA,USA,33.755000,-84.401000,71000
2,BAL,3,Baltimore,Ravens,AFC,North,Baltimore Ravens,7,M&T Bank Stadium,MD,USA,39.277985,-76.622788,71008
3,BUF,4,Buffalo,Bills,AFC,East,Buffalo Bills,1,New Era Field,NY,USA,42.773758,-78.786837,73079
4,CAR,5,Carolina,Panthers,NFC,South,Carolina Panthers,26,Bank of America Stadium,NC,USA,35.225808,-80.852861,73778


In [6]:
attendance = pd.read_csv("Attendance.csv")

attendance['Average attendance'] = attendance['Average attendance'].str.replace(',', '')
attendance.head()

,Team,Metro,Stadium,Home games,Average attendance,Total attendance,Capacity percentage
0,Dallas Cowboys,Dallas,AT&T Stadium,8,91620,"732,958",114.50%
1,New York Jets,New York,MetLife Stadium,8,77982,"623,856",100.00%
2,Green Bay Packers,Green Bay,Lambeau Field,8,77835,"622,677",95.60%
3,New York Giants,New York,MetLife Stadium,8,76941,"615,525",93.30%
4,Denver Broncos,Denver,Broncos Stadium at Mile High,8,76446,"611,571",100.40%


In [7]:
teams_attendance = pd.merge(teams_stadium,
                 attendance[['Team','Average attendance', 'Metro']],
                 left_on='full_name', right_on='Team')
teams_attendance=teams_attendance.drop('Team_y', axis =1 )
teams_attendance.head()

,key,ID,Team_x,Name,conference,division,full_name,stadiumID,stadium_name,state,country,latitude,longitude,capacity,Average attendance,Metro
0,ARI,1,Arizona,Cardinals,NFC,West,Arizona Cardinals,29,U of Phoenix Stadium,AZ,USA,33.527558,-112.263036,63400,62014,Phoenix
1,ATL,2,Atlanta,Falcons,NFC,South,Atlanta Falcons,45,Mercedes-Benz Stadium,GA,USA,33.755000,-84.401000,71000,72898,Atlanta
2,BAL,3,Baltimore,Ravens,AFC,North,Baltimore Ravens,7,M&T Bank Stadium,MD,USA,39.277985,-76.622788,71008,70431,Baltimore
3,BUF,4,Buffalo,Bills,AFC,East,Buffalo Bills,1,New Era Field,NY,USA,42.773758,-78.786837,73079,64962,Buffalo
4,CAR,5,Carolina,Panthers,NFC,South,Carolina Panthers,26,Bank of America Stadium,NC,USA,35.225808,-80.852861,73778,73773,Charlotte


In [8]:
x = pd.merge(out_df,
                            team_df, 
                            on='stadiumID',how = "left")
x =x[x['key'].isna()]
stadium_no_team = x.iloc[:,:7]

In [9]:
stadium_no_team.head()


,stadiumID,stadium_name,state,country,latitude,longitude,capacity
14,14,Qualcomm Stadium,CA,USA,32.783188,-117.119439,70561
23,23,Mall of America Field,MN,US,44.976000,-93.225000,64111
25,25,Georgia Dome,GA,USA,33.757614,-84.400972,74228
28,28,Candlestick Park,CA,US,37.404108,-121.970274,70207
30,30,Edward Jones Dome,MO,USA,38.632975,-90.188547,66000


In [10]:
us_cities = pd.read_csv('us_cities.txt')
top_cities = us_cities[:300]

In [11]:
city_team = pd.merge(teams_attendance,
                 top_cities,
                 left_on='Metro', right_on='city')
city_team=city_team.drop('Metro', axis =1 )


In [12]:
city_team = city_team.drop_duplicates('full_name', keep='first')
city_team

,key,ID,Team_x,Name,conference,division,full_name,stadiumID,stadium_name,state_x,country,latitude,longitude,capacity,Average attendance,rank,city,state_y,population,2000-2013 growth
0,ARI,1,Arizona,Cardinals,NFC,West,Arizona Cardinals,29,U of Phoenix Stadium,AZ,USA,33.527558,-112.263036,63400,62014,6,Phoenix,Arizona,1513367,14.0%
1,ATL,2,Atlanta,Falcons,NFC,South,Atlanta Falcons,45,Mercedes-Benz Stadium,GA,USA,33.755000,-84.401000,71000,72898,40,Atlanta,Georgia,447841,6.2%
2,BAL,3,Baltimore,Ravens,AFC,North,Baltimore Ravens,7,M&T Bank Stadium,MD,USA,39.277985,-76.622788,71008,70431,26,Baltimore,Maryland,622104,-4.0%
3,BUF,4,Buffalo,Bills,AFC,East,Buffalo Bills,1,New Era Field,NY,USA,42.773758,-78.786837,73079,64962,73,Buffalo,New York,258959,-11.3%
4,CAR,5,Carolina,Panthers,NFC,South,Carolina Panthers,26,Bank of America Stadium,NC,USA,35.225808,-80.852861,73778,73773,16,Charlotte,North Carolina,792862,39.1%
5,CHI,6,Chicago,Bears,NFC,North,Chicago Bears,20,Soldier Field,IL,USA,41.862320,-87.616699,61500,61344,3,Chicago,Illinois,2718782,-6.1%
6,CIN,7,Cincinnati,Bengals,AFC,North,Cincinnati Bengals,5,Paul Brown Stadium,OH,USA,39.095309,-84.516003,65535,50754,65,Cincinnati,Ohio,297517,-10.1%
7,CLE,8,Cleveland,Browns,AFC,North,Cleveland Browns,6,FirstEnergy Stadium,OH,USA,41.505885,-81.699458,71516,65765,48,Cleveland,Ohio,390113,-18.1%
8,DAL,9,Dallas,Cowboys,NFC,East,Dallas Cowboys,17,AT&T Stadium,TX,USA,32.747778,-97.092778,80000,91620,9,Dallas,Texas,1257676,5.6%
9,DEN,10,Denver,Broncos,AFC,West,Denver Broncos,13,Broncos Stadium at Mile High,CO,USA,39.743936,-105.020097,76125,76446,22,Denver,Colorado,649495,16.7%


In [13]:
top_50_cities = us_cities[:50]
city_with_team = city_team['city'].tolist()
top_50_cities_city = top_50_cities['city'].tolist()
top_50_cities_no =top_50_cities_city[top_50_cities_city not in city_with_team]

In [14]:
top_50_cities_no = []
for city in top_50_cities_city:
    if city in city_with_team:
        continue
    else: 
        top_50_cities_no.append(city)

In [15]:
top_50_series_no = pd.Series(top_50_cities_no)
top_50_series_no = pd.DataFrame(top_50_series_no).reset_index()
top_50_series_no.columns = ['index','city']
top_50_series_no=top_50_series_no.drop('index', axis =1 )
city_50_no_team = pd.merge(top_50_cities, top_50_series_no,
                          on = 'city')
city_50_no_team = city_50_no_team.drop([2,5,17,18,25]) #Manually removing cities part of metro with teams
city_50_no_team

,rank,city,state,population,2000-2013 growth
0,7,San Antonio,Texas,1409019,21.0%
1,8,San Diego,California,1355896,10.5%
3,11,Austin,Texas,885400,31.7%
4,15,Columbus,Ohio,822553,14.8%
6,19,El Paso,Texas,674433,19.4%
7,20,Memphis,Tennessee,653450,-5.3%
8,27,Oklahoma City,Oklahoma,610613,20.2%
9,28,Louisville/Jefferson County,Kentucky,609893,10.0%
10,29,Portland,Oregon,609456,15.0%
11,30,Las Vegas,Nevada,603488,24.5%


In [16]:
city_team.to_csv('teams_with_stadium.csv')
stadium_no_team.to_csv('stadiums_without_a_team.csv')
city_50_no_team.to_csv('big_cities_without_a_team.csv')